In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
from datetime import datetime
import face_recognition
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt



In [ ]:
# Load LFW dataset (at least 5 images per person)
lfw_people = fetch_lfw_people(min_faces_per_person=5, resize=0.5)

print(f"Total images: {lfw_people.images.shape[0]}")
print("Classes:", lfw_people.target_names)


In [ ]:
known_encodings = []
known_names = []

for i in range(len(lfw_people.images)):
    gray_img = lfw_people.images[i]
    label = lfw_people.target[i]
    name = lfw_people.target_names[label]

    # Convert grayscale to RGB (face_recognition expects 3 channels)
    rgb_img = np.stack([gray_img]*3, axis=-1).astype(np.uint8)

    encodings = face_recognition.face_encodings(rgb_img)
    if encodings:
        known_encodings.append(encodings[0])
        known_names.append(name)


In [ ]:
attendance_file = "attendance.csv"
marked_names = set()

# Create CSV file with headers if not exists
if not os.path.exists(attendance_file):
    with open(attendance_file, "w") as f:
        f.write("Name,Timestamp\n")

        

In [ ]:
video_capture = cv2.VideoCapture(0)

print("Press 'q' to quit")

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for (top, right, bottom, left), encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_encodings, encoding)
        best_match_index = np.argmin(face_distances)

        if matches[best_match_index]:
            name = known_names[best_match_index]

            if name not in marked_names:
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                with open(attendance_file, "a") as f:
                    f.write(f"{name},{timestamp}\n")
                marked_names.add(name)

        # Scale back up face locations since the frame was scaled to 0.5
        top *= 2
        right *= 2
        bottom *= 2
        left *= 2

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.imshow('Intelligent Attendance System', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


In [ ]:
attendance_df = pd.read_csv(attendance_file)
attendance_df.tail()


In [ ]:
# Step 1: Import Libraries
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from datetime import datetime

# Step 2: Load Dataset
faces = fetch_olivetti_faces()
X = faces.data  # Image data
y = faces.target  # Labels (person IDs)

# Step 3: Preprocess Data
# Reshape and normalize the data
X = X.reshape(-1, 64, 64)  # Reshape to 64x64 images
X = X.astype('float32') / 255.0  # Normalize pixel values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Create and Train the ANN Model
model = Sequential()
model.add(Flatten(input_shape=(64, 64)))  # Input layer
model.add(Dense(128, activation='relu'))  # Hidden layer
model.add(Dense(len(np.unique(y)), activation='softmax'))  # Output layer

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed

# Step 5: Face Detection and Recognition
# Initialize the webcam
cap = cv2.VideoCapture(0)

# Function to mark attendance
def mark_attendance(name):
    with open('Attendance.csv', 'a') as f:
        f.write(f'{name},{datetime.now()}\n')

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces_detected = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (64, 64)).reshape(1, 64, 64) / 255.0  # Normalize

        # Predict the person
        prediction = model.predict(face_resized)
        name = np.argmax(prediction)

        # Mark attendance
        mark_attendance(name)

    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
# Step 1: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
import cv2

# Step 2: Load Dataset
faces = fetch_olivetti_faces()
X = faces.data  # Image data
y = faces.target  # Labels (person IDs)

# Step 3: Preprocess Data
# Reshape and normalize the data
X = X.reshape(-1, 64, 64)  # Reshape to 64x64 images
X = X.astype('float32') / 255.0  # Normalize pixel values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Step 4: Create and Train the ANN Model
model = Sequential()
model.add(Flatten(input_shape=(64, 64)))  # Input layer
model.add(Dense(128, activation='relu'))  # Hidden layer
model.add(Dense(len(np.unique(y)), activation='softmax'))  # Output layer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_cat, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Step 6: Face Recognition
# Function to recognize faces from the webcam
def recognize_faces():
    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_detected = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces_detected:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (64, 64)).reshape(1, 64, 64) / 255.0  # Normalize

            # Predict the person
            prediction = model.predict(face_resized)
            name = np.argmax(prediction)

            # Display the recognized name on the frame
            cv2.putText(frame, f'ID: {name}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        cv2.imshow('Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the face recognition function
recognize_faces()

ModuleNotFoundError: No module named 'keras'